In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os, sys
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import statsmodels.stats.api as sms

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
path = '/nfs/turbo/umms-awaljee/wsliu/Data/NRD/'

In [3]:
module_path = '/home/wsliu/Codes/DLproj'
if module_path not in sys.path:
    sys.path.append(module_path)
if module_path+'/NRD' not in sys.path:
    sys.path.append(module_path+'/NRD')

In [40]:
from ccs_tools import core_dtypes_pd
from utils import preprocess

Using TensorFlow backend.


## Prepare hyper-parameters and generate the .sh files

In [4]:
!rm hypertune*.sh

In [10]:
cohorts = ['ami']
tst_seeds = range(3)
eval_datas = ['tst']
resample_fracs = [1., 0.1, 0.03]

In [11]:
para_itr = itertools.product(cohorts, tst_seeds, eval_datas, resample_fracs)
para_lst = list(para_itr)

In [12]:
len(para_lst)

9

In [13]:
n_jobs = 8
for para, job_ind in zip(para_lst, itertools.cycle(range(n_jobs))):
    with open('hypertune'+str(job_ind)+'.sh', 'a') as f:
        f.write('python recycle_pred.py --cohort {} --tst_seed {} --eval_data {} --resample_frac {} --job_index {}\n'.format(*para, job_ind))

In [107]:
job_index = 5
for para in para_lst:
    with open('hypertune'+str(job_index)+'.sh', 'a') as f:
        f.write('python train_template_all0827.py --model_name {0} --code_embed_dim {1} --fc_width {2} --md_width {3} --lr1 {4} --lr2 {5} --dropout {6} --batchsize {7} --embed_file {8} --tst_seed {9} --cohort {10} --sep_dx1 {11} --val_fold {12} --result_file {13} --job_index {14}\n'.format(*para, job_index))

## Inference

In [20]:
!rm hypertune*.sh

rm: cannot remove ‘hypertune*.sh’: No such file or directory


In [26]:
cohorts = ['ami']
split_seeds = [0, 1]
train_idxs = [1, 2]
sample_seeds = range(10)
n_vals = [20]

In [27]:
para_itr = itertools.product(cohorts, split_seeds, train_idxs, sample_seeds, n_vals)
para_lst = list(para_itr)

In [28]:
len(para_lst)

40

In [22]:
n_jobs = 4
for para, job_ind in zip(para_lst, itertools.cycle(range(n_jobs))):
    with open('hypertune'+str(job_ind)+'.sh', 'a') as f:
        f.write('python inference.py --cohort {} --split_seed {} --train_idx {} --sample_seed {} --n_val {} --job_index {}\n'.format(*para, job_ind))

In [107]:
job_index = 5
for para in para_lst:
    with open('hypertune'+str(job_index)+'.sh', 'a') as f:
        f.write('python train_template_all0827.py --model_name {0} --code_embed_dim {1} --fc_width {2} --md_width {3} --lr1 {4} --lr2 {5} --dropout {6} --batchsize {7} --embed_file {8} --tst_seed {9} --cohort {10} --sep_dx1 {11} --val_fold {12} --result_file {13} --job_index {14}\n'.format(*para, job_index))

## Result Analysis

In [2]:
res = pd.DataFrame()